In [25]:
import os
import torch
import torchvision.models as tvm
from torchvision import transforms
from sklearn.cluster import KMeans
import numpy as np
from PIL import Image

In [26]:
def print_count_files(dir, message = None):
    if os.path.isdir(dir):
        file_count = sum(1 for _ in os.listdir(dir) if os.path.isfile(os.path.join(dir, _)))
        print(f"{message if message else dir}: {file_count}")


def get_all_files(dir):
    if os.path.isdir(dir):
        files = []
        for root, directories, filenames in os.walk(dir):
            for filename in filenames:
                files.append(os.path.join(root, filename))
        return files
    return []



dirs = ["./q4/patch_camelyon/train/0", "./q4/patch_camelyon/train/1", "./q4/patch_camelyon/test",]    
msgs = ["train with Label 0", "train with label 1", "test images"] 

for dir, msg in zip(dirs, msgs):
    print_count_files(dir, msg)


image_lists = [get_all_files(x) for x in dirs]




train with Label 0: 10500
train with label 1: 5500
test images: 2000


In [27]:
base_resin = tvm.resnet50(pretrained=True)
base_resin = torch.nn.Sequential(*list(base_resin.children())[:-1])

for p in base_resin.parameters():
    p.requires_grad = False


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


/opt/anaconda3/envs/sut-default-env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/sut-default-env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [30]:
def health_check(msg, i, threshold = 100):
    if (i % threshold == 0):
            print(msg)

def image_cleanup(image_list):
    images = []
    for (i, image_path) in enumerate(image_list):
        health_check(f"loading {i}th image", i)
        image = Image.open(image_path)
        image.convert("RGB")
        # print(image_path)
        image = transform(image)
        images.append(image)
    feats = []
    with torch.no_grad():
        for (i, img) in enumerate(images):
            health_check(f"extracting {i}th image features")
            img = img.unsqueeze(0)
            feature = base_resin(img)
            feature = feature.squeeze().numpy()
            feats.append(feature)

    n_clusters = 2
    km = KMeans(n_clusters=n_clusters)
    labels = km.fit_predict(feats)

    # Determine the cluster with the lower number of items
    label_counts = np.bincount(labels)
    lower_cluster_label = np.argmin(label_counts)

    # Get the names of images assigned to the lower cluster label
    assigned_images = [image_list[i] for i, label in enumerate(labels) if label == lower_cluster_label]

    return assigned_images

print(len(image_lists[2]))
image_cleanup(image_lists[2])

2000
loading 0th image
loading 100th image
loading 200th image
loading 300th image
loading 400th image
loading 500th image
loading 600th image
loading 700th image
loading 800th image
loading 900th image
loading 1000th image
loading 1100th image
loading 1200th image
loading 1300th image
loading 1400th image
loading 1500th image
loading 1600th image
loading 1700th image
loading 1800th image
loading 1900th image
